In [3]:
import cutword
import pandas as pd
import time
import models_function as mf
import cutword
import string
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
import pymysql
from joblib import dump, load

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
page = pd.read_csv('page.csv', index_col=False)
page = page.drop(columns="Unnamed: 0")
page

,公司代號,公司簡稱,發言日期,發言時間,主旨
0,2371,大同,109/06/23,22:38:22,代子公司中華映管股份有限公司公告一○九年股東常會 重要決議事項(假決議)
1,8929,富堡,109/06/23,22:37:54,公告本公司第三屆薪資報酬委員會委員任期屆滿
2,1260,富味鄉,109/06/23,22:04:23,公告本公司收到臺北關通知繳納罰鍰及稅款公文
3,1260,富味鄉,109/06/23,22:03:29,補充公告本公司與富鼎產業科技股份有限公司間和解事件 所涉訴訟案件進度
4,5203,訊連,109/06/23,21:31:27,公告本公司董事會決議委任新任薪酬委員
...,...,...,...,...,...
867,3073,天方能源,109/06/20,07:00:02,公告本公司名稱由「凱柏實業股份有限公司」 更名為「天方能源科技股份有限公司」
868,3441,聯一光,109/06/20,07:00:02,主旨:公告本公司名稱由「聯一光學工業股份有限公司」 更名為「聯一光電股份有限公司」
869,4152,台微體,109/06/21,11:00:48,安畢黴獲得上市後批量放大變更核准
870,3073,天方能源,109/06/21,07:00:02,公告本公司名稱由「凱柏實業股份有限公司」 更名為「天方能源科技股份有限公司」


In [5]:

my_tokenizer = load('/Users/wangyuda/Desktop/文字探勘/Program_Final_test/models_function_box/my_tokenizer.joblib')

##########################################################################################

# 網站登入資訊，帳號與密碼
id = "p314008dsa1"
password = "MaTuItgeeu30hy$D63"

# 輸入我們的WordPress網址，若試著連上該網址，應該會出現「XML-RPC server accepts POST requests only.」才對。
url = "http://ec2-52-87-157-212.compute-1.amazonaws.com//xmlrpc.php"

# 新文章要直接發布的話，就不用改，如果要變成草稿，就改成"draft"
which = "publish"

# 建立客戶端，登入網頁
wp = Client(url, id, password)

##########################################################################################

for i in range(len(page)):
    add_new = []
    cut_new = []
    add_new_col = []
    for j in range(5):
        add_new_col.append(page.iloc[i, j])

    add_new.append(add_new_col)
    cut_new.append(page.iloc[i, 4])

    add_new_df = pd.DataFrame(data = add_new, columns = ["公司代號", "公司簡稱", "發言日期", "發言時間", "主旨"])
    cut_new_df = pd.DataFrame(data = cut_new, columns = ["cut_new"])

    cut = cutword.ckiptoken(cut_new_df)
    cut_0 = cut.content_tokenized
    cut = cut.iloc[0, 0]
    token = mf.preprocess_text(my_tokenizer, cut_0)
    
    # 將input 輸入模型，給出預測
    intensity = mf.predict_intensity(token)
    big_event = mf.predict_Big_Event(token)
    small_event = mf.predict_Small_Event(token)
    up_or_down = mf.predict_Month_abnormal_returns(token)
    
    intensity = intensity[0][0]
    
    # 若事件強度絕對值大於1，則上傳到網站
    if abs(intensity) > 1:
        
    ##########################################################################################
    
        # 設定輸出內容
        title = add_new_df.iloc[0, 2] + ' ' + add_new_df.iloc[0, 1] 
        excerpt = "事件強度：" + str(intensity) + '\n' + "預期漲跌：" +  up_or_down
        category = str(add_new_df.iloc[0, 0]) + ' ' + str(add_new_df.iloc[0, 1])
        content = excerpt + '\n' + "發言時間：" + add_new_df.iloc[0, 3] + '\n' + "大事件類別：" + big_event[0][0] + '\n'
        content = content + "小事件類別：" + small_event[0][0] + '\n' + "新聞內容：" + add_new_df.iloc[0, 4]

        # 依據讀取的資料，在部落格中建立新文章
        post = WordPressPost()
        post.post_status = which
        post.excerpt = excerpt
        post.title = title
        post.content = content
        post.terms_names = {"category": [category]}

        wp.call(NewPost(post))
    
    ##########################################################################################
    
    # 連接資料庫
    db = pymysql.connect("127.0.0.1","root","s70399S80228","sys" )
    
    cursor = db.cursor()
    
    a = int(add_new_df.iloc[0,0])
    b = add_new_df.iloc[0,1]
    c = add_new_df.iloc[0,2]
    d = add_new_df.iloc[0,3]
    e = int(intensity)
    f = up_or_down
    g = big_event[0][0]
    h = small_event[0][0]
    i_1 = add_new_df.iloc[0,4]
    j = cut
    
    sql = """INSERT INTO result(Stock_id, Company_name, Date ,Time,
       Intensity, Direction, Big_event_type, Small_event_type, News_content, News_cut)
       VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
    try:
        # Execute the SQL command
        cursor.execute(sql, (a, b, c, d, e, f, g, h, i_1, j))
#         print('Successful')
        # Commit your changes in the database
        db.commit()
    except:
        # Rollback in case there is any error
        print('Failed')
        print(i)
        db.rollback()
    
    db.close()


In [35]:
# 存csv檔
# page = pd.concat([page, page2], axis=0, ignore_index=True)
page.to_csv(path_or_buf="page.csv")

# 放入過去的資料做驗證

In [ ]:
import pandas as pd 
import numpy as np
import sklearn
import keras
from keras import models
from keras import layers
from keras.layers import LSTM
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score #The F-Measure is a popular metric for imbalanced classification.
from sklearn.metrics import confusion_matrix

In [6]:
test_data = pd.read_excel('/Users/wangyuda/Desktop/文字探勘/Program_Final_test/2019+2020年新聞資訊事件(5.28).xlsx')

In [ ]:
#     data = {
#         'Stock_id' : add_new_df.iloc[0,0],
#         'Company_name' : add_new_df.iloc[0,1],
#         'Date' : add_new_df.iloc[0,2],
#         'Time' : add_new_df.iloc[0,3],
#         'Intensity' : intensity,
#         'Direction' : up_or_down,
#         'Big_event_type' : big_event[0][0],
#         'Small_enent_type' : small_event[0][0],
#         'News_content' : add_new_df.iloc[0,4],
#         'News_cut' : cut
#     }
    
#     table = 'result'
    
#     keys = ', '.join(data.keys())
#     values = ', '.join(['%s'] * len(data))
    
#     sql = 'INSERT INTO {table}({keys}) VALUES ({values})'.format(table=table, keys=keys, values=values)

#     try:
#         cursor.execute(sql, tuple(data.values()))
#         print('Successful')
#         db.commit()

#     except:
#         print('Failed')
#         db.rollback()

In [7]:
test_data 

,個股代號,公司簡稱,事件日,TCRI(年/月),事件強度,大事件類別,小事件類別,事件內容,Unnamed: 8,content_tokenized
0,1218,泰山,20190101,6(2018/09),0,M_經營層,MT06_高管異動,發言人林俐婉內部調動，由江巍峰接任。。,19.0,發言人 林俐婉 內部 調動 由 江巍峰 接任
1,1503,士電,20190101,4(2018/09),0,M_經營層,MT06_高管異動,內部稽核主管林志強內部調動，由莊文清接任。。,22.0,內部 稽核 主管 林志強 內部 調動 由 莊文清 接任
2,1504,東元,20190101,4(2018/09),0,M_經營層,MT06_高管異動,會計主管藍俊雄內部調動，由林鴻名接任。。,20.0,會計 主管 藍俊雄 內部 調動 由 林鴻名 接任
3,1709,和益,20190101,5(2018/09),0,M_經營層,MT06_高管異動,內部稽核主管游本詮內部調動，由曾筱茜接任。。,22.0,內部 稽核 主管 游本詮 內部 調動 由 曾筱茜 接任
4,1721,三晃,20190101,7(2018/09),0,M_經營層,MT06_高管異動,財務經理洪廷宜內部調動，由王婷渝接任。。,20.0,財務 經理 洪廷宜 內部 調動 由 王婷渝 接任
...,...,...,...,...,...,...,...,...,...,...
36712,2609,陽明,20200330,7(2019/09),-1,I_產業前景,IS01_營收變動或客戶/商品/通路策略,貨櫃船運市場往年農曆年後二到三周的淡季砍班措施，今年因新冠肺炎疫情，已連續八周砍班，合計砍掉...,NaN,貨櫃 船運 市場 往年 農曆年 後 二 到 三 周 的 淡季 砍 班 措施 今年 因 新冠 ...
36713,2881,富邦金,20200330,,1,I_產業前景,IF01_增資/減資/融資發債,"富邦金擬發行無擔保普通公司債(富邦金109-1)，以募集台幣40,000,000(仟元)，用...",NaN,富邦金擬 發行 無 擔保 普通 公司債 富邦金 1091 以 募集 台幣 40000000仟...
36714,2882,國泰金,20200330,,1,I_產業前景,IF01_增資/減資/融資發債,"國泰金擬發行無擔保普通公司債(國泰金109-1)，以募集台幣30,000,000(仟元)，用...",NaN,國泰金擬 發行 無 擔保 普通 公司 債國 泰金 1091 以 募集 台幣 30000000...
36715,2888,新光金,20200330,,1,I_產業前景,IF01_增資/減資/融資發債,"新光金擬發行無擔保普通公司債(新光金109-1)，以募集台幣3,000,000(仟元)，用以...",NaN,新光 金擬 發行 無 擔保 普通 公司債 新光金 1091 以 募集 台幣 3000000仟...


In [12]:
test_data .iloc[1, 7]

'內部稽核主管林志強內部調動，由莊文清接任。。'

In [ ]:
# for i in range(len(page)):

my_tokenizer = load('/Users/wangyuda/Desktop/文字探勘/Program_Final_test/models_function_box/my_tokenizer.joblib')

##########################################################################################
result = []
for i in range(len(test_data)):
# for i in range(1000):
#     add_new = []
    cut_new = []
#     add_new_col = []
#     for j in range(5):
#         add_new_col.append(page.iloc[i, j])

#     add_new.append(add_new_col)
    cut_new.append(test_data.iloc[i, 7])

#     add_new_df = pd.DataFrame(data = add_new, columns = ["公司代號", "公司簡稱", "發言日期", "發言時間", "主旨"])
    cut_new_df = pd.DataFrame(data = cut_new, columns = ["cut_new"])

    cut = cutword.ckiptoken(cut_new_df)
    cut_0 = cut.content_tokenized
    cut = cut.iloc[0, 0]
    token = mf.preprocess_text(my_tokenizer, cut_0)
    
    # 將input 輸入模型，給出預測
    intensity = mf.predict_intensity(token)
    big_event = mf.predict_Big_Event(token)
    small_event = mf.predict_Small_Event(token)
    up_or_down = mf.predict_Month_abnormal_returns(token)
    
    ##########################################################################################
    
    a = int(test_data.iloc[i, 0])
    b = test_data.iloc[i, 1]
    c = test_data.iloc[i, 2]
    e = int(intensity)
    f = up_or_down
    g = big_event[0][0]
    h = small_event[0][0]
    i = test_data.iloc[i, 7]
    j = cut
    
    result.append([a, b, c, e, f, g, h, i, j])
    
    
result_df = pd.DataFrame(data = result, columns = ["Stock_id", "Company_name", "Date",
                "Intensity", "Direction", "Big_event_type", "Small_event_type", "News_content", "News_cut"])


In [33]:
result_df

,Stock_id,Company_name,Date,Intensity,Direction,Big_event_type,Small_event_type,News_content,News_cut
0,1218,泰山,20190101,0,下跌,M_經營層,MT06_高管異動,發言人林俐婉內部調動，由江巍峰接任。。,發言人 林俐婉 內部 調動 由 江巍峰 接任
1,1503,士電,20190101,0,下跌,M_經營層,MT06_高管異動,內部稽核主管林志強內部調動，由莊文清接任。。,內部 稽核 主管 林志強 內部 調動 由 莊文清 接任
2,1504,東元,20190101,0,上漲,M_經營層,MT06_高管異動,會計主管藍俊雄內部調動，由林鴻名接任。。,會計 主管 藍俊雄 內部 調動 由 林鴻名 接任
3,1709,和益,20190101,0,下跌,M_經營層,MT06_高管異動,內部稽核主管游本詮內部調動，由曾筱茜接任。。,內部 稽核 主管 游本詮 內部 調動 由 曾筱茜 接任
4,1721,三晃,20190101,0,下跌,M_經營層,MT06_高管異動,財務經理洪廷宜內部調動，由王婷渝接任。。,財務 經理 洪廷宜 內部 調動 由 王婷渝 接任
...,...,...,...,...,...,...,...,...,...
95,2809,京城銀,20190103,-1,下跌,I_產業前景,IS01_營收變動或客戶/商品/通路策略,"2018年12月累計營收7,000,815千元，年減22%。2018年12月單月營收588,...",2018年 12月 累計 營收 7000815千 元 年減 222018年 12月 單月 營...
96,2867,三商壽,20190103,-1,下跌,M_經營層,經營層_其他,"經理人本人胡志誠於2019/01/03預計申讓1,000張股票，預計申讓比重16%，交易方式...",經理人 本人 胡志誠 於 20190103 預計 申讓 1000 張 股票 預計 申讓 比重...
97,2913,農林,20190103,0,下跌,M_經營層,MT06_高管異動,發言人岡友峰內部調動，由黃翠香接任。。,發言人 岡友峰 內部 調動 由 黃翠香 接任
98,3066,李洲,20190103,-1,下跌,I_產業前景,IS01_營收變動或客戶/商品/通路策略,"2018年12月累計營收408,535千元，年減9%。2018年12月單月營收33,223千...",2018年 12月 累計 營收 408535千 元 年減 92018年 12月 單月 營收 ...


In [ ]:
Intensity_y = data_df["事件強度"]
preds_Intensity = predict_intensity(x)

accuracy = accuracy_score(Intensity_y, preds_Intensity)
precision = precision_score(Intensity_y, preds_Intensity,average=None) # average=None: The precision scores for each class are returned
recall = recall_score(Intensity_y, preds_Intensity,average=None)
f1 = f1_score(Intensity_y, preds_Intensity,average=None)
c_matrix = confusion_matrix(Intensity_y, preds_Intensity)
print('accuracy: '+str(accuracy)+'\n')
print('precision: '+str(precision)+'\n')
print('recall: '+str(recall)+'\n')
print('F1 score: '+str(f1)+'\n')
print('Confusion matrix: ')
print(c_matrix)

In [41]:
result_df.to_excel('/Users/wangyuda/Desktop/文字探勘/Program_Final_test/test.xlsx')


In [47]:
page

,公司代號,公司簡稱,發言日期,發言時間,主旨
0,2371,大同,109/06/23,22:38:22,代子公司中華映管股份有限公司公告一○九年股東常會 重要決議事項(假決議)
1,8929,富堡,109/06/23,22:37:54,公告本公司第三屆薪資報酬委員會委員任期屆滿
2,1260,富味鄉,109/06/23,22:04:23,公告本公司收到臺北關通知繳納罰鍰及稅款公文
3,1260,富味鄉,109/06/23,22:03:29,補充公告本公司與富鼎產業科技股份有限公司間和解事件 所涉訴訟案件進度
4,5203,訊連,109/06/23,21:31:27,公告本公司董事會決議委任新任薪酬委員
...,...,...,...,...,...
867,3073,天方能源,109/06/20,07:00:02,公告本公司名稱由「凱柏實業股份有限公司」 更名為「天方能源科技股份有限公司」
868,3441,聯一光,109/06/20,07:00:02,主旨:公告本公司名稱由「聯一光學工業股份有限公司」 更名為「聯一光電股份有限公司」
869,4152,台微體,109/06/21,11:00:48,安畢黴獲得上市後批量放大變更核准
870,3073,天方能源,109/06/21,07:00:02,公告本公司名稱由「凱柏實業股份有限公司」 更名為「天方能源科技股份有限公司」
